In [11]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib as plt
from wrangle import wrangle
from acquire import acquire
from prepare import prepare, clean

In [14]:
pd.set_option("display.max_rows", None, "display.max_columns", None) 

pd.reset_option("display.max_rows", "display.max_columns")

In [61]:
df = acquire()

65 – 80 = Commodity Coffee The type used to make supermarket coffee, blends and instant.

80+ = Specialty Coffee The flavours are more subtle, the cup more balanced. At Pact we only buy coffees of 84+, though most of them score 86+.

90+ = Presidential Award These prestigious coffees make up less than 1% of the specialty coffee market, they’re just that rare. And that good.

# Clean Notes

#### Dropped Columns

These columns got dropped because they didn't offer any value
- 'Unnamed: 0'
- 'Owner'
- 'Farm.Name'
- 'Company'
- 'Expiration'
- 'Lot.Number'
- 'Mill'
- 'Producer'
- 'Certification.Address'
- 'Certification.Contact'
- 'ICO.Number'
- 'Certification.Body'
- 'In.Country.Partner'
- 'Owner.1'
- 'Species'
- 'Altitude'
- 'unit_of_measurement'
- 'altitude_low_meters' 
- 'altitude_high_meters'

#### Handle Nulls

Dropped any row that was missing a value from the following columns:
- 'Altitude'
- 'Region'
- 'Quakers'
- 'Harvest.Year'
- 'Variety'

Used the median value to fill in missing values from the following columns:
- 'Color'
- 'Processing.Method'

#### Convert Data Types

Changed the following columns data types:
- 'Bag.Weight' -> float
- 'Grading.Date' -> datetime
- 'Harvest.Year' -> int

#### Miscellaneous

- In 'Color' I Converted any Bluish-Green value to Blue-Green
- In 'Bag.Weight' converted lbs into kg
- Seperated 'Grading.Date' into three new columns 'grading_month', 'grading_year', 'grading_day'

#### Rename

Changed the following columns names to something more conventional:
- 'Country.of.Origin' -> 'country'
- 'Region' -> 'region'
- 'Number.of.Bags' -> 'number_of_bags'
- 'Harvest.Year' -> 'harvest_year', 
- 'Grading.Date' -> 'grading_date'
- 'Variety' -> 'variety'
- 'Processing.Method' -> 'processing_method'
- 'Aroma' -> 'aroma',
- 'Flavor' -> 'flavor'
- 'Aftertaste' -> 'aftertaste'
- 'Acidity' -> 'acidity'
- 'Body' -> 'body'
- 'Balance' -> 'balance'
- 'Uniformity' -> 'uniformity'
- 'Clean.Cup' -> 'clean_cup'
- 'Sweetness' -> 'sweetness'
- 'Cupper.Points' -> 'cupper_points',
- 'Total.Cup.Points' -> 'total_cup_points'
- 'Moisture' -> 'moisture'
- 'Category.One.Defects' -> 'category_one_defects', 
- 'Quakers' -> 'quakers'
- 'Color' -> 'color'
- 'Category.Two.Defects' -> 'category_two_defects'

#### Engineered columns

- 'grading_month' = from Grading.Date
- 'grading_year' = from Grading.Date
- 'grading_day' = from Grading.Date
- 'excellent_rating' = from total_cup_points >= 84
- 'total_bag_weight' = number_of_bags * bag_weight

#### Encode
- Created dummy columns for - 'variety', 'processing_method', 'color'


In [ ]:
def clean(df):
    '''
    Takes in a pandas dataframe
    Designed to clean the 'arabica_data_cleaned.csv' file
    Dropped columns that didn't offer any value.
    Dropped any row that was missing a value from - Altitude, Region, Quakers, Harvest.Year, Variety
    Used the median value to fill in missing values from - Color, Processing.Method
    Convert Data Types
    Miscellaneous fixes
    Renamed columns
    returns a pandas dataframe
    '''
    
    # Dropped Columns
    col_remove = ['Unnamed: 0', 'Owner', 'Farm.Name', 'Company', 'Expiration','Lot.Number', 'Mill', 'Producer', 'Certification.Address', 'Certification.Contact', 'ICO.Number','Certification.Body','In.Country.Partner', 'Owner.1', ]
    df = df.drop(columns=col_remove)
    
    # Dropped any row that was missing a value from - Altitude, Region, Quakers, Harvest.Year, Variety
    df = df[~df['Altitude'].isnull()]
    df = df[~df['Region'].isnull()]
    df = df[~df['Quakers'].isnull()]
    df = df[~df['Harvest.Year'].isnull()]
    df = df[~df['Variety'].isnull()]
    
    # Used the median value to fill in missing values for - Color, Processing.Method
    df['Color'] = np.where(df['Color'] == 'None', 'Green', df.Color)
    df['Color'] = np.where(df['Color'].isnull(), 'Green', df.Color)
    df['Color'] = np.where(df['Color'] == 'Bluish-Green', 'Blue-Green', df.Color)
    df['Processing.Method'] = np.where(df['Processing.Method'].isnull(), 'Washed / Wet', df['Processing.Method'])

    # Convert Data Type of Bag.Weight and change from lbs to Kilograms
    df[['bag_weight', 'bag_weight_unit']]= df['Bag.Weight'].str.split(expand=True)
    df.bag_weight = df.bag_weight.astype(float)
    df.bag_weight = np.where(df.bag_weight_unit == 'lbs', round(df.bag_weight * 2.20462), df.bag_weight)
    df = df.drop(columns=['bag_weight_unit', 'Bag.Weight'])
    
    # Convert Data Type for Grading.Date and split by day,month, year 
    df['Grading.Date'] = pd.to_datetime(df['Grading.Date'])
    df['grading_month'] = df['Grading.Date'].dt.month
    df['grading_year'] = df['Grading.Date'].dt.year
    df['grading_day'] = df['Grading.Date'].dt.day

    # Miscellaneous fixes
    
    # Dropped this value of Harvest.Year because it equalled 'Mayo a Julio'
    df = df.drop(list(df[df['Harvest.Year']=='Mayo a Julio'].index))
    
    # Anywhere where given two years as a range for Harvest.Year I filled with grading year instead
    df['Harvest.Year'] = np.where(df['Harvest.Year'].str.contains('/'), df.grading_year, df['Harvest.Year'])
    df['Harvest.Year'] = np.where(df['Harvest.Year'].str.contains('-'), df.grading_year, df['Harvest.Year'])
    df['Harvest.Year'] = df['Harvest.Year'].astype(int)
    
    # Data Input Errors, used google to compare regions altitude to confirm correct altitude
    df.at[543,'altitude_mean_meters']=1100
    df.at[896, 'altitude_mean_meters']=1901.64
    df.at[1040, 'altitude_mean_meters']=1100
    df.at[1144, 'altitude_mean_meters']=1901.64
    df.at[41, 'altitude_mean_meters']=1150
    df.at[42, 'altitude_mean_meters']=1150
    
    # Removing older verisons or now un-needed columns
    more_col_remove = ['Species', 'Altitude', 'unit_of_measurement', 'altitude_low_meters', 'altitude_high_meters']
    df = df.drop(columns=more_col_remove)
    
    # Renamed Columns
    df = df.rename(columns={'Country.of.Origin':'country', 'Region':'region', 'Number.of.Bags':'number_of_bags', 'Harvest.Year':'harvest_year', 
                   'Grading.Date':'grading_date', 'Variety':'variety', 'Processing.Method':'processing_method', 'Aroma':'aroma',
                   'Flavor':'flavor', 'Aftertaste':'aftertaste', 'Acidity':'acidity', 'Body':'body', 'Balance':'balance',
                   'Uniformity':'uniformity', 'Clean.Cup':'clean_cup', 'Sweetness':'sweetness', 'Cupper.Points':'cupper_points',
                   'Total.Cup.Points':'total_cup_points', 'Moisture':'moisture', 'Category.One.Defects':'category_one_defects', 
                   'Quakers':'quakers', 'Color':'color', 'Category.Two.Defects': 'category_two_defects'})
    
    # Created Target Feature, Looking for total_cup_points greater than 85
    df['excellent_rating'] = np.where(df.total_cup_points > 85, 1, 0)
    
    return df
    

In [ ]:
def encode(df):
    '''
    Encode takes in a pandas dataframe
    Creates dummy columns for variety, processing_method, & color
    Adds the new columns to pandas dataframe
    Drops non numeric columns
    Returns a pandas dataframe
    '''
    
    # Making Dummy Columns for variety, processing_method, & color
    dummy_columns = ['variety', 'processing_method', 'color']
    df_dummy = pd.get_dummies(df[dummy_columns])
    
    # Droppign Blue-Green from color since there are only two colors, its either green or its not
    df_dummy = df_dummy.drop(columns=['color_Blue-Green'])
    
    # Combine our dummy columns with our data frame
    df = pd.concat([df, df_dummy], axis = 1)
    
    # Drop any non numeric columns
    df = df.drop(columns=cols)
    
    return df

# Cleaning Work Below

In [ ]:
# acquired not cleaned
df = acquire()

In [ ]:
# got the data dictionary working
get_data_dictionary(df).head()

In [ ]:
df.head(3)

In [ ]:
df.describe().T

# Clean

In [ ]:
df.head(1)

In [ ]:
col_remove = ['Unnamed: 0', 'Owner', 'Farm.Name', 'Company', 'Lot.Number', 'Mill', 'Producer', 'Certification.Address', 'Certification.Contact', 'ICO.Number','Certification.Body','In.Country.Partner','unit_of_measurement', 'Owner.1', ]

In [ ]:
df.head(1)

In [ ]:
df[(~df['Region'].isnull()) & (df['Altitude'].isnull())]

Anything not labeled in lbs or kg must be dropped

In [ ]:
df[df['Country.of.Origin'] == 'United States (Hawaii)'].Altitude.value_counts()

In [ ]:
df.head(3)

## Trying to salvage empty Altitude rows by looking at farm name and region of other observations

In [ ]:
df[(~df['Farm.Name'].isnull()) & (df['Altitude'].isnull())][['Farm.Name', 'Altitude']]

In [ ]:
# kona missing altitude
kona_df = df[df['Region']=='kona']
kona_df[['Farm.Name', 'Altitude', 'Region']]['Farm.Name'].value_counts()

In [ ]:
df = df[~df['Altitude'].isnull()]

In [ ]:
kona_df[kona_df[['Farm.Name', 'Altitude']]['Farm.Name']== 'kona pacific farmers cooperative']

### Salvage not possible, I will be Removing all altitude missing values

# looking at missing color values

In [ ]:
# 142 missing values
# Need to rename Bluish-Green with just Blue-Green
# fill in NA with Green

In [ ]:
len(df[df['Color'].isnull()])

In [ ]:
df['Color'].value_counts(dropna=False)

In [ ]:
df['Color'] = np.where(df['Color'] == 'Bluish-Green', 'Blue-Green', df.Color)

In [ ]:
df['Color'] = np.where(df['Color'] == 'None', 'Green', df.Color)
df['Color'] = np.where(df['Color'].isnull(), 'Green', df.Color)

In [ ]:
df[df['Color'].isnull()]

# Processing.Method

In [ ]:
df['Processing.Method'].value_counts(dropna=False)

In [ ]:
df['Processing.Method'] = np.where(df['Processing.Method'].isnull(), 'Washed / Wet', df['Processing.Method'])

# Variety

In [ ]:
# acquired not cleaned
df = acquire()
df.Variety.value_counts(dropna=False)

df = df[~df['Variety'].isnull()]

In [ ]:
df.head()

In [ ]:
df = acquire()
df.head(1)

In [ ]:
df.columns[df.isnull().sum() > 0]

In [ ]:
df = df[~df['Region'].isnull()]
df = df[~df['Quakers'].isnull()]
df = df[~df['Harvest.Year'].isnull()]

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

# Working with Bag Weight

In [ ]:
df['Bag.Weight'].value_counts(dropna = False)

In [ ]:
df[['bag_weight', 'bag_weight_unit']]= df['Bag.Weight'].str.split(expand=True)

In [ ]:
df[['bag_weight', 'bag_weight_unit', 'Bag.Weight']]

In [ ]:
df.bag_weight_unit.value_counts()

In [ ]:
df.bag_weight = df.bag_weight.astype(float)

In [ ]:
df.bag_weight.dtype

In [ ]:
df.bag_weight = np.where(df.bag_weight_unit == 'lbs', round(df.bag_weight * 2.20462), df.bag_weight)

In [ ]:
df[['bag_weight', 'bag_weight_unit', 'Bag.Weight']][df.bag_weight_unit == 'lbs']

In [ ]:
df = acquire()
df.head(1)

In [ ]:
# Splitting up the Grading Date
df['Grading.Date'] = pd.to_datetime(df['Grading.Date'])

In [ ]:
df['grading_month'] = df['Grading.Date'].dt.month

In [ ]:
df['grading_year'] = df['Grading.Date'].dt.year

In [ ]:
df['grading_day'] = df['Grading.Date'].dt.day

In [ ]:
df = df.drop(list(df[df['Harvest.Year']=='Mayo a Julio'].index))
len(df)

In [ ]:
df['Harvest.Year'] = np.where(df['Harvest.Year'].str.contains('/'), df.grading_year, df['Harvest.Year'])
df['Harvest.Year'] = np.where(df['Harvest.Year'].str.contains('-'), df.grading_year, df['Harvest.Year'])

In [ ]:
df = acquire()

In [ ]:
df.info()

In [ ]:
df[((df['altitude_low_meters'] + df['altitude_high_meters']) / 2) != df['altitude_mean_meters']]

In [ ]:
# Convert altitude feet to meters (3.28084)
df.unit_of_measurement.value_counts()

In [ ]:
#df['altitude_mean'] = np.where(df.unit_of_measurement == 'ft', round(df.altitude_mean_meters * 3.28084), df.altitude_mean_meters)

In [ ]:
df[df['altitude_mean_meters'] > 4000]

In [ ]:
# Few typos and after some googling the typos add up
df.at[543,'altitude_mean_meters']=1100
df.at[896, 'altitude_mean_meters']=1901.64
df.at[1040, 'altitude_mean_meters']=1100
df.at[1144, 'altitude_mean_meters']=1901.64

In [ ]:
df['altitude_mean_meters'].plot()

In [ ]:
df.head(1)

In [ ]:
df = acquire()
df = clean(df)
df.head(1)

In [ ]:
df['total_bag_weight'] = df['number_of_bags'] * df['bag_weight']

In [ ]:
df[['total_bag_weight', 'number_of_bags', 'bag_weight']]

In [ ]:
df.rename(columns={'Country.of.Origin':'country', 'Region':'region', 'Number.of.Bags':'number_of_bags', 'Harvest.Year':'harvest_year', 
                   'Grading.Date':'grading_date', 'Variety':'variety', 'Processing.Method':'processing_method', 'Aroma':'aroma',
                   'Flavor':'flavor', 'Aftertaste':'aftertaste', 'Acidity':'acidity', 'Body':'body', 'Balance':'balance',
                   'Uniformity':'uniformity', 'Clean.Cup':'clean_cup', 'Sweetness':'sweetness', 'Cupper.Points':'cupper_points',
                   'Total.Cup.Points':'total_cup_points', 'Moisture':'moisture', 'Category.One.Defects':'category_one_defects', 
                   'Quakers':'quakers', 'Color':'color', 'Category.Two.Defects': 'category_two_defects'})

In [ ]:
df.head()

In [ ]:
df = acquire()

In [ ]:
df = clean(df)

In [ ]:
df.head()

In [ ]:
df['excellent_rating'] = np.where(df.total_cup_points > 85, 1, 0)

In [ ]:
df.head()

In [ ]:
 df.total_cup_points.median(),  df.total_cup_points.max()

In [ ]:
len(df[df.total_cup_points < 80])

In [ ]:
len(df)

In [ ]:
df.at[41, 'altitude_mean_meters']=1150
df.at[42, 'altitude_mean_meters']=1150
df.head()

# Notes

- Some bag weights are in kg others in lbs, Anything not labeled in lbs or kg must be dropped
- Some altitude low meters are in meters some in ft

# Encoded / Create Dummies

In [ ]:
df = acquire()
df = clean(df)

In [ ]:
df.head(1)
cols =  ['country', 'region', 'grading_date', 'variety', 'processing_method', 'color', 'grading_month', 'grading_year', 'grading_day']

In [ ]:
dummy_columns = ['variety', 'processing_method', 'color']

In [ ]:
df_dummy = pd.get_dummies(df[dummy_columns])

In [ ]:
df_dummy = df_dummy.drop(columns=['color_Blue-Green'])

In [ ]:
df = pd.concat([df, df_dummy], axis = 1)

In [ ]:
df = acquire()
df = clean(df)
df = encode(df)

# If i had more time i would have created a continent column and created dummies for that too

In [2]:
# variety, and cleaning method, maybe elavation (bottom end)

In [46]:
def remove_outliers(df):
    df = df[(df.processing_method != 'Other') & (df.processing_method != 'Pulped natural / honey')]
    
    df = df[df.variety != 'Catimor']
    df = df[df.variety != 'SL14']
    df = df[df.variety != 'SL28']
    df = df[df.variety != 'Pacas']
    df = df[df.variety != 'Gesha']
    df = df[df.variety != 'Pacamara']
    df = df[df.variety != 'SL34']
    df = df[df.variety != 'Java']
    df = df[df.variety != 'Ethiopian Yirgacheffe']
    df = df[df.variety != 'Blue Mountain']
    df = df[df.variety != 'Ruiru 11']
    df = df[df.variety != 'Ethiopian Heirlooms']
    df = df[df.variety != 'Sumatra Lintong']
    df = df[df.variety != 'Sumatra']
    df = df[df.variety != 'Pache Comun']
    df = df[df.variety != 'Mandheling']
    df = df[df.variety != 'Marigojipe']
    df = df[df.variety != 'Arusha']
    df = df[df.variety != 'Moka Peaberry']
    df = df[df.variety != 'Peaberry']
    
    return df

In [12]:
df = acquire()
df = clean(df)

In [15]:
df.head(1)

,country,region,number_of_bags,harvest_year,grading_date,variety,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,cupper_points,total_cup_points,moisture,category_one_defects,quakers,color,category_two_defects,altitude_mean_meters,bag_weight,grading_month,grading_year,grading_day,excellent_rating
1,Ethiopia,guji-hambela,300,2014,2015-04-04,Other,Washed / Wet,8.75,8.67,8.5,8.58,8.42,8.42,10.0,10.0,10.0,8.58,89.92,0.12,0,0.0,Green,1,2075.0,60.0,4,2015,4,1


In [17]:
df.processing_method.value_counts()

Washed / Wet                 724
Natural / Dry                171
Semi-washed / Semi-pulped     53
Other                         25
Pulped natural / honey         9
Name: processing_method, dtype: int64

In [26]:
len(df)

982

In [27]:
df = df[(df.processing_method != 'Other') & (df.processing_method != 'Pulped natural / honey')]

In [29]:
len(df)

948

In [44]:
df.variety.value_counts()[df.variety.value_counts() < 25]

Catimor                  18
SL14                     16
SL28                     13
Pacas                    13
Gesha                    12
Pacamara                  8
SL34                      7
Java                      2
Ethiopian Yirgacheffe     2
Blue Mountain             2
Ruiru 11                  2
Ethiopian Heirlooms       1
Sumatra Lintong           1
Sumatra                   1
Pache Comun               1
Mandheling                1
Marigojipe                1
Arusha                    1
Moka Peaberry             1
Peaberry                  1
Name: variety, dtype: int64

In [ ]:
df = df[(df.variety != 'Catimor')
df = df[(df.variety != 'SL14')
df = df[(df.variety != 'SL28')
df = df[(df.variety != 'Pacas')
df = df[(df.variety != 'Gesha')
df = df[(df.variety != 'Pacamara')
df = df[(df.variety != 'SL34')
df = df[(df.variety != 'Java')
df = df[(df.variety != 'Ethiopian Yirgacheffe')
df = df[(df.variety != 'Blue Mountain')
df = df[(df.variety != 'Ruiru 11')
df = df[(df.variety != 'Ethiopian Heirlooms')
df = df[(df.variety != 'Sumatra Lintong')
df = df[(df.variety != 'Sumatra')
df = df[(df.variety != 'Pache Comun')
df = df[(df.variety != 'Mandheling')
df = df[(df.variety != 'Marigojipe')
df = df[(df.variety != 'Arusha')
df = df[(df.variety != 'Moka Peaberry')
df = df[(df.variety != 'Peaberry')

In [48]:
df = acquire()
df = clean(df)
df = remove_outliers(df)

In [49]:
df.variety.value_counts()

Caturra           229
Bourbon           203
Typica            199
Other              93
Catuai             65
Yellow Bourbon     28
Mundo Novo         27
Name: variety, dtype: int64

In [50]:
df.processing_method.value_counts()

Washed / Wet                 645
Natural / Dry                150
Semi-washed / Semi-pulped     49
Name: processing_method, dtype: int64

In [51]:
len(df)

844